In [1]:
import requests
import json
import pandas as pd
from time import gmtime, strftime
from sqlalchemy import create_engine

class Inmueble:
    mercadolibre_id = 'MLA79242'
    tabla = 'mercadolibre_inmueble' # Utilizada para la carga en SQL
    columnas = ['id', 'fecha', 'producto_latitud', 'producto_longitud', 'url', 'tipo_propiedad']

    # Conversion de json a dataframe y rearmado de datos para que sea formato tabla
    def adapt(self, items):
        data = pd.DataFrame(items)
        data = data.groupby(['id']).first().reset_index() # Elimina duplicados
        largo = data.shape[0]

        for i in range(0, largo):
            try:
                # La fecha es la fecha de ejecucion, como este es un proceso diario, la necesitamos
                # para poder identificar las diferentes corridas
                data.loc[i, 'fecha'] = strftime("%d/%m/%Y", gmtime())
                data.loc[i, 'mercadolibre_id'] = data.loc[i, 'id']
                data.loc[i, 'producto_latitud'] = data.loc[i, 'location']['latitude']
                data.loc[i, 'producto_longitud'] = data.loc[i, 'location']['longitude']
                data.loc[i, 'url'] = data.loc[i, 'permalink']

                data_attr = pd.DataFrame(data.loc[i, 'attributes'])
                property_type = data_attr.loc[data_attr['id'] == 'PROPERTY_TYPE']['value_name'].tolist()
                data.loc[i, 'tipo_propiedad'] = property_type[0] if property_type else 'DESCONOCIDO'

            except Exception as e:
                print(f'Error adaptando: {i} - {data.loc[i, "mercadolibre_id"]} - {e}')

        data = data.fillna('') # Los nulos los completamos con un string vacio
        print(data[self.columnas])
        return data[self.columnas]

class MercadoLibreAPI:
    debug = False # Limita el procesamiento a los primeros elementos del primer indice, habilida la salida por pantalla de mensajes
    query = None
    objeto = None
    ml_url = 'http://api.mercadolibre.com/sites/MLA/search?category='
    items = []

    def set_debug(self, debug):
        self.debug = debug
        if self.debug:
            print("Modo Debug ENCENDIDO")

    def request_get(self, url):
        if self.debug:
            print("Procesando url:", url)
        try:
            return requests.get(url).json()
        except Exception as e:
            print(f'Error en la solicitud GET: {url} - {e}')
            return None

    def search(self, objeto):
        self.objeto = objeto()
        url = f'{self.ml_url}{self.objeto.mercadolibre_id}&_PublishedToday_YES'
        print("Buscando:", url)

        if self.debug:
            paginators = 5
        else:
            total_items = self.request_get(url).get('paging', {}).get('total', 0)
            paginators = min(round(total_items / 50) + 1, 200)

        for offset in range(paginators):
            url = f'{self.ml_url}{self.objeto.mercadolibre_id}&_PublishedToday_YES&limit=50&offset={offset*50}'
            jsdata = self.request_get(url)

            if jsdata is not None:
                self.items += jsdata.get('results', [])

        self.adapt()

    def adapt(self):
        o = self.objeto
        self.items = o.adapt(self.items)

    def export(self, tipo='csv'):
        if tipo.lower() == 'sql':
            self.export_sql()
        elif tipo.lower() == 'csv':
            self.export_csv()
        else:
            print("No existe el metodo de exportacion:", tipo.lower())

    def export_sql(self):
        if not self.items.empty:
            engine = create_engine('sqlite:///mercadolibre_data.db')  # Cambia la URL según tu base de datos
            self.items.to_sql(name=self.objeto.tabla, con=engine, if_exists='replace', index=False)
            print(f'Datos exportados a la tabla {self.objeto.tabla} en la base de datos.')


    def export_csv(self):
        archivo = f"indice_{self.objeto.mercadolibre_id}.csv"
        if self.debug:
            print("Guardando archivo", archivo)
        self.items.to_csv(archivo, sep=";", decimal=".")

ml = MercadoLibreAPI()
ml.set_debug(True)
ml.search(Inmueble)
ml.export()
ml.export_sql()


Modo Debug ENCENDIDO
Buscando: http://api.mercadolibre.com/sites/MLA/search?category=MLA79242&_PublishedToday_YES
Procesando url: http://api.mercadolibre.com/sites/MLA/search?category=MLA79242&_PublishedToday_YES&limit=50&offset=0
Procesando url: http://api.mercadolibre.com/sites/MLA/search?category=MLA79242&_PublishedToday_YES&limit=50&offset=50
Procesando url: http://api.mercadolibre.com/sites/MLA/search?category=MLA79242&_PublishedToday_YES&limit=50&offset=100
Procesando url: http://api.mercadolibre.com/sites/MLA/search?category=MLA79242&_PublishedToday_YES&limit=50&offset=150
Procesando url: http://api.mercadolibre.com/sites/MLA/search?category=MLA79242&_PublishedToday_YES&limit=50&offset=200
Error adaptando: 70 - MLA1387429836 - 'latitude'
                id       fecha producto_latitud producto_longitud  \
0    MLA1105492775  22/11/2023       -32.934677        -60.670467   
1    MLA1110183439  22/11/2023       -34.689981        -58.626694   
2    MLA1130074501  22/11/2023       -